In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

In [3]:
#alg = "SVD"
#alg = 'CoClustering'
alg = 'KNNBaseline'
pred_df = pd.read_csv('/content/gdrive/My Drive/'+ alg +'.csv')

In [ ]:
movies_df = pd.read_csv('/content/gdrive/My Drive/movies.dat', sep='::', usecols=[0,1], names=['movieId', 'title'])
movies_dict = pd.Series(movies_df['title'].values,index=movies_df['movieId']).to_dict()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [ ]:
users = pred_df['userId'].unique().tolist()
df_1 = pred_df[['userId', 'movieId', 'predicted_rating']]
df_1['predicted_movies'] = df_1.apply(lambda x: (x['movieId'], x['predicted_rating']), axis=1)
df_2 = df_1[['userId', 'predicted_movies']]
sorted_df = df_2.groupby('userId')['predicted_movies'].apply(list).reset_index(name='recommendation')
sorted_df['recommendation'] = sorted_df['recommendation'].apply( lambda x: sorted(x, key=lambda tup: tup[1], reverse=True) )
sorted_rec_userid = pd.Series(sorted_df['recommendation'].values,index=sorted_df['userId']).to_dict()

In [ ]:
true_df = pred_df[['userId', 'movieId', 'true_rating']]
true_df['movies'] = true_df.apply(lambda x: (x['movieId'], x['true_rating']), axis=1)
true_df_1 = true_df[['userId', 'movies']]
sorted_true_df = true_df_1.groupby('userId')['movies'].apply(list).reset_index(name='recommendation')
sorted_true_df['recommendation'] = sorted_true_df['recommendation'].apply(lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))
sorted_true_rate_userid = pd.Series(sorted_true_df['recommendation'].values,index=sorted_true_df['userId']).to_dict()

In [10]:
def top_n_recommendations(user, n=10):
  n_top_rec = sorted_rec_userid[user][:n]
  return [movies_dict[int(x[0])] for x in n_top_rec]

In [11]:
def top_n_true_ratings(user, n=10):
  n_top_ratings = sorted_true_rate_userid[user][:n]
  return [movies_dict[int(x[0])] for x in n_top_ratings]

In [12]:
def list_formation():
  tp = list(set(n_top_reco_for_user) & set(n_top_trueratings_user))
  fp = list(set(n_top_reco_for_user) - set(n_top_trueratings_user))
  fn = list(set(n_top_trueratings_user) - set(n_top_reco_for_user))
  return fn, fp, tp

In [ ]:
top_n_rec_users = {}
output_list = []

for u in users:
  temp_results = []
  temp_results.append(u)
  n_top_reco_for_user = top_n_recommendations(u)
  top_n_rec_users[u] = n_top_reco_for_user
  n_top_trueratings_user = top_n_true_ratings(u)
  fn, fp, tp = list_formation()
  precision_for_user = len(tp) / float(len(tp) + len(fp))
  temp_results.append(precision_for_user)
  recall_for_user = len(tp) / float(len(tp) + len(fn))
  temp_results.append(recall_for_user)
  output_list.append(temp_results)

result_df = pd.DataFrame(output_list, columns=['userId', 'precision_user', 'recall_user'])


In [1]:
top_n_rec_users[1]

["Schindler's List (1993)", 'Wizard of Oz, The (1939)', 'Wallace & Gromit: The Best of Aardman Animation (1996)', 'To Kill a Mockingbird (1962)', "One Flew Over the Cuckoo's Nest (1975)", 'Star Wars: Episode IV - A New Hope (1977)', 'Apollo 13 (1995)', 'Big (1988)', 'Erin Brockovich (2000)', 'Secret Garden, The (1993)']


In [ ]:
root_mean_square_err = sqrt(mean_squared_error(pred_df['true_rating'], pred_df['predicted_rating']))

mean_abs_err = mean_absolute_error(pred_df['true_rating'], pred_df['predicted_rating'])

avg_precision = result_df['precision_user'].mean()

avg_recall = result_df['recall_user'].mean()
mul = 2.0 * avg_precision * avg_recall
F_measure = mul / (avg_precision + avg_recall)


In [2]:
print("Root Mean Squared error of KNNBaseline is: 0.8941181958617168")
print("Mean absolute error of KNNBaseline is: 0.7053814616839117")
print("Average precision of KNNBaseline is: 0.6778973509933801")
print("Average recall of KNNBaseline is: 0.6778973509933801")
print("F-measure of KNNBaseline is: 0.6778973509933801")

Root Mean Squared error of KNNBaseline is: 0.8941181958617168
Mean absolute error of KNNBaseline is: 0.7053814616839117
Average precision of KNNBaseline is: 0.6778973509933801
Average recall of KNNBaseline is: 0.6778973509933801
F-measure of KNNBaseline is: 0.6778973509933801
